In [ ]:
import requests
from time import sleep

def fetch_sidewalks_and_nodes(bbox):
    overpass_url = "http://overpass-api.de/api/interpreter"
    
    # Overpass query to fetch sidewalks and all their referenced nodes
    #  (>;) recurses down to get all nodes referenced by the ways
    overpass_query = f"""
    [out:json];
    (
      way["highway"="footway"]["footway"~"sidewalk|crossing"]({bbox});
    );
    out body;
    >;
    out body;
    """

    
    # Send the request to the Overpass API
    response = requests.get(overpass_url, params={'data': overpass_query})
    
    if response.status_code == 200:
        return response.json()
    else:
        print(response)
        print("Error fetching data from Overpass API")
        return None

def is_self_enclosing(nodes):
    if not nodes or len(nodes) < 3:  # Need at least 3 nodes for a closed way
        return False
    # Get the coordinates of the first and last node
    first_node = nodes[0]
    last_node = nodes[-1]
    
    # Check if the first and last node coordinates are the same
    return first_node['lat'] == last_node['lat'] and first_node['lon'] == last_node['lon']

def process_sidewalks(bbox):
    data = fetch_sidewalks_and_nodes(bbox)
    
    if not data:
        return []
    
    # Create a dictionary to store node details by ID for quick lookup
    nodes_dict = {}
    ways_list = []
    
    # Process all elements from the response
    for element in data['elements']:
        if element['type'] == 'node':
            # Store node details
            nodes_dict[element['id']] = {
                'lat': element['lat'],
                'lon': element['lon']
            }
        elif element['type'] == 'way':
            # Store way for later processing
            ways_list.append(element)
    
    # Process ways and check if they are self-enclosing
    closed_ways = []
    for way in ways_list:
        node_ids = way['nodes']
        node_details = []
        for node_id in node_ids:
            if node_id in nodes_dict:
                node_details.append(nodes_dict[node_id])
            else:
                print(f"Warning: Node ID {node_id} not found in fetched nodes")
        
        # Check if the way is self-enclosing
        if is_self_enclosing(node_details):
            closed_ways.append(way['id'])
    
    return closed_ways

# Usage
# bbox = "40.607619,-74.081014,40.61053,-74.0752"
bbox = "40.749,-73.835,40.775,-73.795"  
closed_sidewalk_ways = process_sidewalks(bbox)
print(f"\nFound {len(closed_sidewalk_ways)} self-enclosing sidewalk ways: {closed_sidewalk_ways}")


Found 11 self-enclosing sidewalk ways: [813002121, 865682134, 866673781, 877372924, 882696407, 892395428, 892395429, 893099586, 893389756, 917499891, 917499892]


In [44]:
for way in closed_sidewalk_ways:
    print(f'https://www.openstreetmap.org/way/{way}')

https://www.openstreetmap.org/way/813002121
https://www.openstreetmap.org/way/865682134
https://www.openstreetmap.org/way/866673781
https://www.openstreetmap.org/way/877372924
https://www.openstreetmap.org/way/882696407
https://www.openstreetmap.org/way/892395428
https://www.openstreetmap.org/way/892395429
https://www.openstreetmap.org/way/893099586
https://www.openstreetmap.org/way/893389756
https://www.openstreetmap.org/way/917499891
https://www.openstreetmap.org/way/917499892
